<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fabián Castro`

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

D:\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Cargar datos
path = '../datasets/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.sample(5)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
896,es_0513988,product_es_0741008,reviewer_es_0076669,1,Ofrece mucho ruido de ventiladores y poca pote...,Puff,es,video_games
985,es_0184814,product_es_0181891,reviewer_es_0762935,1,"De rosa no tiene nada, parece cuarzo blanco.","No es rosa, es blanco",es,jewelry
97,es_0750013,product_es_0094489,reviewer_es_0742598,1,Me parece un engaño al 100% Pagas 4 euros más ...,Engañado con la carátula que dice ser en español,es,video_games
160,es_0915319,product_es_0774444,reviewer_es_0228027,1,Horrible!! Quedaron conmigo en que lo enviaban...,Horrible,es,jewelry
44,es_0093446,product_es_0474284,reviewer_es_0475129,3,"Buen precio, pero algo mas pequeño de lo que m...",pequeño,es,jewelry


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
def pre_process(text):
    text = text.lower()
    text = re.sub(r'[\W\d_]+', ' ', text)
    return [word for word in text.split() if word not in stopwords]

In [4]:
data['pp'] = data['review_body'].apply(lambda body: pre_process(body))
data.sample(5)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
985,es_0184814,product_es_0181891,reviewer_es_0762935,1,"De rosa no tiene nada, parece cuarzo blanco.","No es rosa, es blanco",es,jewelry,"[rosa, parece, cuarzo, blanco]"
817,es_0899444,product_es_0380236,reviewer_es_0550330,4,"La expansión va en un código aparte del disco,...",Expansión en Código y no añadido en disco físico,es,video_games,"[expansión, va, código, aparte, disco, ademas,..."
253,es_0476924,product_es_0032366,reviewer_es_0513955,3,Son bastante pesados.,Pasables,es,jewelry,"[bastante, pesados]"
28,es_0066993,product_es_0287018,reviewer_es_0956389,1,Era el regalo de Papá Noel para mi hija y vení...,Una gran decepción,es,video_games,"[regalo, papá, noel, hija, venía, perfectament..."
226,es_0007527,product_es_0239150,reviewer_es_0752987,4,Es como espera bien de calidad aunque me queda...,Pulsera tibetana,es,jewelry,"[espera, bien, calidad, aunque, queda, grande,..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [5]:
#Representación en forma de diccionario
dictionary = Dictionary(documents = data['pp'].values)
print(f'Se han procesado {dictionary.num_pos} palabras')
print(f'Se han procesado {dictionary.num_nnz} palabras distintas')

Se han procesado 12459 palabras
Se han procesado 11891 palabras distintas


In [6]:
dictionary.filter_extremes(no_below = 5,
                           no_above = .5)
corpus = [dictionary.doc2bow(text) for text in data['pp'].values]
corpus[:1]

[[(0, 1), (1, 2), (2, 1), (3, 1)]]

In [7]:
data['pp'].head(1)

0    [buen, buena, calidad, buena, presentación]
Name: pp, dtype: object

In [8]:
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 6, passes = 200)
model.print_topics(num_words = 8)

[(0,
  '0.057*"calidad" + 0.040*"regalo" + 0.036*"buena" + 0.033*"precio" + 0.027*"pulsera" + 0.026*"buen" + 0.022*"bonito" + 0.020*"plata"'),
 (1,
  '0.038*"bien" + 0.034*"juego" + 0.024*"si" + 0.023*"caja" + 0.019*"precio" + 0.017*"así" + 0.016*"calidad" + 0.015*"mejor"'),
 (2,
  '0.038*"juego" + 0.027*"producto" + 0.025*"perfecto" + 0.019*"si" + 0.019*"viene" + 0.018*"solo" + 0.017*"entrega" + 0.017*"bien"'),
 (3,
  '0.044*"día" + 0.034*"juego" + 0.030*"llegado" + 0.022*"pedido" + 0.018*"amazon" + 0.017*"después" + 0.015*"días" + 0.015*"llegar"'),
 (4,
  '0.038*"bien" + 0.035*"foto" + 0.028*"bonito" + 0.019*"queda" + 0.018*"grande" + 0.017*"cumple" + 0.017*"switch" + 0.016*"poner"'),
 (5,
  '0.029*"si" + 0.027*"bien" + 0.022*"producto" + 0.022*"mas" + 0.020*"calidad" + 0.019*"puede" + 0.018*"precio" + 0.018*"mando"')]

###  `[25 pts]` Punto 3: Visualización de LDA

In [9]:
# Resultados
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(lda_display)

> Nótese el cúmulo de temas en el centro, donde 3 distintas agrupaciónes (1, 3, 4) se intersectan las unas con las otras.

In [10]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None
    
for t in range(0, 6):
    top_name = f'topic_{t}'
    data[top_name] = data['pp'].apply(lambda doc: get_doc_top_n(doc, t))

In [11]:
# Documentos más representativos de cada tópico
for t in range(0, 6):
    print(f'*********************************************************** TOPIC {t} *******************************************************')
    topic = f'topic_{t}'
    for i, article in enumerate(data.sort_values(topic, ascending = False)['review_body'].values[:2]): #.values para una lista
        print(f'Artículo {i}')
        print(article[:500])
        print()
    print()

*********************************************************** TOPIC 0 *******************************************************
Artículo 0
Un buen colgante de buena calidad, y de un gran envío rápido. La única pega es que las dos cadenas se lían entre sí muchas veces, es elegante, pero siempre se están liando entre sí, por lo demás bien.

Artículo 1
compre esta pulsera por probar y ha sido toda una grata sorpresa, por un precio baratisimo tiene una mas q aceptable calidad y es muy bonita, tal cual como aparece en la foto


*********************************************************** TOPIC 1 *******************************************************
Artículo 0
Es un juegazo que merece muchísimo la pena tener, con varios añadidos que lo hacen si cabe aún más interesante. Esta versión de switch contiene música orquestada ( la calidad se nota MUCHÍSIMO ), nuevas historias y contenido adicional que ayudan a entender ciertas partes de la historia mejor, la posibilidad de acelerar el combate ( este a

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

> Una base de datos puede ser muy útil para aplicar diferentes modelos y realizar análisis desde múltiples perspectivas. Por ejemplo para el caso de los ejercicios de modelos no supervisados, le presenta al científico estructuras o patrónes que a simple vista son difíciles de detectar. Pero por otro lado, los ejercicios de clasificación, le presentan al científico que existen estructuras que él esperaría encontrar en los datos.